# Enigma Machine Design



## Inside the Enigma

The enigma machine is a fairly complex encryption machine which consists of four main sections:

![major-components](../images/mission-x-challenge/intro-inside-enigma.png)



# Enigma Machine Code

## The Keyboard

The keyboard is used to retrieve the user input. The Enigma machine is a symmetric encryption machine. Which means that it can be used to both encrypt or decrypt a message using the same settings. The keyboard is hence used to either enter the plaintext that needs to be encrypted or the ciphertext that needs to be decrypted.

They keyboard consists of 26 keys for each letter of the alphabet. This means that encrypted messages will be joined up without any spaces or punctuation signs.

Notice how the keyboard starts with the letters `QWERTZ` instead of `QWERTY`. This is due to the fact that in the German language, the letter Z is more often used than the letter.

In [ ]:
import string
import random

class Letter:
    """Letter class : contains the string representation of a letter and its position in the alphabet"""
    def __init__(self, y) -> None:
        if isinstance(y, str):
            self.letter = y.lower()
            self.index = Letter.a2i(self.letter)
        elif isinstance(y, int):
            self.index = y
            self.letter = Letter.i2a(y)
        else:
            raise ValueError("Not Implemented")

    def __add__(self, y):
        q = self.get_type(y)
        x = (26 + self.index + q) % 26
        return Letter(string.ascii_lowercase[x])

    def __sub__(self, y):
        q = self.get_type(y)
        x = (26 + self.index - q) % 26
        return Letter(string.ascii_lowercase[x])

    def __eq__(self, y):
        return self.letter == y.letter

    def __str__(self):
        return self.letter

    def get_type(self, y):
        if isinstance(y, Letter):
            q = y.index
        else:
            q = y
        return q

    @staticmethod
    def random():
        return Letter(random.choice(string.ascii_lowercase))

    @staticmethod
    def a2i(a: str) -> int:
        return string.ascii_lowercase.find(a)

    @staticmethod
    def i2a(i: int) -> str:
        return string.ascii_lowercase[i]


class Pair:
    """A Plugboard pair"""
    def __init__(self, A: str, B: str) -> None:
        self.a = Letter(A)
        self.b = Letter(B)

    def get(self, x: Letter) -> Letter:
        if self.a == x:
            return self.b
        elif self.b == x:
            return self.a
        else:
            return None

    def __str__(self):
        return f"{self.a}{self.b}"

In [ ]:

def test_letter_from_str():
    tests = []
    for i in range(26):
        character = string.ascii_lowercase[i]
        index = i
        tests.append({"letter": character, "index": index})

    for test in tests:
        letter = Letter(test["letter"])
        assert letter.letter == test["letter"] and letter.index == test["index"]


def test_letter_from_index():
    tests = []
    for i in range(26):
        character = string.ascii_lowercase[i]
        index = i
        tests.append({"letter": character, "index": index})

    for test in tests:
        letter = Letter(test["index"])
        assert letter.letter == test["letter"] and letter.index == test["index"]


def test_pairs():
    tests = [
        {"pair": {"A": "x", "B": "c"}, "off": "t"},
        {"pair": {"A": "r", "B": "t"}, "off": "g"},
        {"pair": {"A": "r", "B": "t"}, "off": "l"},
    ]

    for test in tests:
        p = test["pair"]
        pair = Pair(p["A"], p["B"])

        a = Letter(p["A"])
        b = Letter(p["B"])
        off = Letter(test["off"])

        assert(pair.get(a) == b and pair.get(b) == a and pair.get(off) == None)


test_letter_from_str()
test_letter_from_index()
test_pairs()



## The plugboard

Once a key is pressed on the keyboard, it goes through the plugboard which provides the first stage of the encryption process. It is based on the principles of a substitution cipher, a form of transposition encryption.

To setup the keyboards, short wires are used to connect pairs of letters that will be permuted. For instance on the picture below the letter W will be replaced with a D and the letter D with a W as a (red) wire is used to connect these two letters/plugs. Similarly, letter V will become letter Z and Z will become V.

In a code book the plugboard settings would be recorded as follows: DW VZ

![plugboard](../images/mission-x-challenge/intro-plugboard.jpeg)


In [ ]:
class Plugboard:
    def __init__(self, pairs: str) -> None:
        pairs = pairs.split(" ")
        self.pairs = [Pair(*p) for p in pairs if p]

    def encode(self, x: Letter):
        for pair in self.pairs:
            ret = pair.get(x)
            if ret:
                return ret
        return x

    def active_len(self):
        return len(self.pairs)

    def randomize(self, pairs=10):
        alphabet = string.ascii_lowercase
        while self.active_len() < pairs:
            a = random.choice(alphabet)
            alphabet = alphabet.replace(a, "")
            b = random.choice(alphabet)
            alphabet = alphabet.replace(b, "")
            self.pairs.append(Pair(a, b))

    def __str__(self):
        return " ".join(map(lambda p: str(p), self.pairs))

In [ ]:
def test_plugboard_init():
    tests = [
        {
            "sequence": "AB CD EF",
            "len": 3
        }
    ]

    for test in tests:
        p = Plugboard(test["sequence"])

        assert(p.active_len() == test["len"])

def test_plugboard_init_random():
    p = Plugboard("")
    p.randomize()

    assert(p.active_len() == 10)

test_plugboard_init()
test_plugboard_init_random()




## The Rotors

After the plugboard, the letter goes through the three rotors in order (from right to left), each of them changing it differently using a combination of transposition cipher and Caesar cipher! On the Enigma M3 there are three rotor slots and five rotors to choose from. Each rotor is identified using a Roman numeral from I to V. This provides a few settings of the Enigma machine: which rotors to use, and in which order to position them. In a code book this setting would be recorded as IV II III (Left, Middle and Right rotors).

Each of the five rotors encrypt the letter differently using a transposition/permutation cipher and can be connected in the Enigma machine with a different Ring setting. Another setting is the initial position of the rotors: Which letters are you going to set each rotor to begin with (e.g. A/B/C../Z sometimes recorded in a codebook using numbers (01 for A, 02 for B up to 26 for Z). This creates a Caesar Shift (Caesar Cipher). On an Enigma machine, you can change the position of the rotors by turning the three wheels.

Different versions of Enigma (e.g. M4) included four rotors which made the encryption process and the number of possible settings even bigger.

On our Enigma M3 emulator, you can click on the rotors to access the Enigma rotors settings:

![rotors](../images/mission-x-challenge/intro-rotors.jpeg)

![settings](../images/mission-x-challenge/intro-settings.jpeg)


In [ ]:
class Rotor:
    """Enigma rotor class"""

    def __init__(
        self,
        name: str,
        wiring: str,
        turnovers: str,
        reflector=False,
        position="A",
        ringstellung="A",
        rotate=True,
    ) -> None:
        self.name = name
        self.wiring = [Letter(wire) for wire in wiring]
        self.turnovers = [Letter(turnover) for turnover in turnovers]
        self.position = Letter(position)
        self.ringstellung = Letter(ringstellung)
        self.reflector = reflector
        self.can_rotate = rotate and not reflector

    def randomize(self):
        """Randomize the machine position"""
        self.position = Letter.random()
        self.ringstellung = Letter.random()

    def encode(self, letter: Letter):
        """From the plugboard to the reflector"""
        wiring_letter = letter + self.position - self.ringstellung
        offset = self.wiring[wiring_letter.index] - self.position + self.ringstellung
        return offset

    def decode(self, letter: Letter):
        """From the reflector to the plugboard"""
        wiring_letter = letter + self.position - self.ringstellung
        offset = Letter(self.wiring.index(wiring_letter)) - self.position + self.ringstellung
        return offset

    def rotate(self):
        """Rotate one time this rotor"""
        if not self.reflector and self.can_rotate:
            self.position += 1

    def __str__(self):
        return (f"{self.name}\n"
            f"Turnover: {''.join([str(t) for t in self.turnovers])}\n"
            f"Wiring: {''.join([str(w) for w in self.wiring])}\n"
            f"Position: {self.position}\n")


class Rotors:
    # M3 
    class M3:
        ETW     = Rotor("ETW", "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "", rotate=False)
        I       = Rotor("I", "EKMFLGDQVZNTOWYHXUSPAIBRCJ", "Q")
        II      = Rotor("II", "AJDKSIRUXBLHWTMCQGZNPYFVOE", "E")
        III     = Rotor("III", "BDFHJLCPRTXVZNYEIWGAKMUSQO", "V")
        IV      = Rotor("IV", "ESOVPZJAYQUIRHXLNFTGKDCMWB", "J")
        V       = Rotor("V", "VZBRGITYUPSDNHLXAWMJQOFECK", "Z")
        VI      = Rotor("VI", "JPGVOUMFYQBENHZRDKASXLICTW", "ZM")
        VII     = Rotor("VII", "NZJHGRCXMYSWBOUFAIVLPEKQDT", "ZM")
        VIII    = Rotor("VIII", "FKQHTLXOCBJSPDZRAMEWNIUYGV", "ZM")
        UKWB    = Rotor("UKWB", "YRUHQSLDPXNGOKMIEBFZCWVJAT", "", reflector=True)
        UKWC    = Rotor("UKWC", "FVPJIAOYEDRZXWGCTKUQSBNMHL", "", reflector=True)

    class M4(M3):
        Beta    = Rotor("Beta", "LEYJVCNIXWPBQMDRTAKZGFUHOS", "", rotate=False)
        Gamma   = Rotor("Gamma", "FSOKANUERHMBTIYCWLQPZXVGJD", "", rotate=False)
        UKWB    = Rotor("UKWB", "ENKQAUYWJICOPBLMDXZVFTHRGS", "", reflector=True)
        UKWC    = Rotor("UKWC", "RDOBJNTKVEHMLFCWZAXGYIPSUQ", "", reflector=True)



In [ ]:
def test_init_rotor():
    Rotor = Rotors.M3.I
    print(Rotor)


test_init_rotor()


## The Reflector

The reflector is another type of rotor inside the machine. Once the letter has gone through the three rotors from right to left, the reflector will reflect the electrical current back through the rotors, sending the encrypted letter through the rotors from left to right for another 3 stages of encryption and then through the plugboard again for a final substitution cipher. When going through the reflector, a permutation cipher is also applied to the letter.

Different versions of reflectors were used on different versions of Enigma machines. Each reflector would apply a different permutation cipher. Enigma M3 machines were equipped with either a UKW-B or UKW-C reflector. You can apply these two reflectors in the rotor settings window of our emulator (see screenshot above).

The diagram below shows the journey of a letter through the encryption process of an Enigma M3.

![reflector](../images/mission-x-challenge/intro-reflector.png)

In [ ]:
def test_init_reflector():
    Reflector = Rotors.M3.UKWB
    print(Reflector)

test_init_reflector()


## The lampboard
The lampboard is the final stage of the encryption process and is used to show the output (encrypted letter). It consists of 26 light bulbs, one for each letter of the alphabet.

## The Enigma Machine 

In [ ]:
class Machine:
    def __init__(
        self, name: str, rotors: list, plugboard: Plugboard, double_stepping=True
    ) -> None:
        self.name = name
        self.rotors = rotors
        self.plugboard = plugboard
        self.double_stepping = double_stepping

    def encode(self, sequence: str, split=5, debug=False) -> str:
        ret = ""
        for x in sequence:
            if x.lower() in string.ascii_lowercase:
                letter = Letter(x)
                j = [letter]

                # plugboard
                letter = self.plugboard.encode(letter)
                j.append(letter)

                self.rotate()

                # one way
                for rotor in self.rotors:
                    letter = rotor.encode(letter)
                    j.append(letter)

                # other way
                for rotor in reversed(self.rotors):
                    if not rotor.reflector:
                        letter = rotor.decode(letter)
                        j.append(letter)

                letter = self.plugboard.encode(letter)
                j.append(letter)

                ret += letter.letter

                # print(self)
                if debug:
                    print('->'.join(str(l) for l in j))

        if split > 0:
            ret = " ".join(ret[i : i + split] for i in range(0, len(ret), split))
        return ret

    def rotate(self):
        if self.double_stepping:
            self.rotate_double_stepping()
        else:
            self.rotate_normal()

    def rotate_double_stepping(self):
        rotors = list(filter(lambda r: r.can_rotate, self.rotors))
        assert len(rotors) == 3

        if self.rotors[2].position in self.rotors[2].turnovers:
            self.rotors[1].rotate()
            self.rotors[2].rotate()
            self.rotors[3].rotate()
        elif self.rotors[1].position in self.rotors[1].turnovers:
            self.rotors[1].rotate()
            self.rotors[2].rotate()
        else:
            self.rotors[1].rotate()

    def rotate_normal(self):
        for r, rotor in enumerate(self.rotors):
            if r == 0:
                rotor.rotate()

    def reset(self):
        for rotor in self.rotors:
            rotor.position = Letter("A")

    def set_position(self, position: list):
        position = list(reversed(position))
        assert len(position) == len(self.rotors)
        for rotor, p in zip(self.rotors, position):
            rotor.position = Letter(p)
    
    def get_position(self) -> list:
        positions = list(reversed(list(map(lambda r: r.position, filter(lambda r: r.can_rotate, self.rotors)))))
        return positions

    def set_ringstellung(self, ringstellung: list):
        ringstellung = list(reversed(ringstellung))
        assert len(ringstellung) == len(self.rotors)
        for rotor, p in zip(self.rotors, ringstellung):
            rotor.ringstellung = Letter(p)

    def get_ringstellung(self) -> list:
        return list(reversed(map(lambda r: r.ringstellung, self.rotors)))

    def set_rotor_position(self, name: str, position: str):
        for rotor in self.rotors:
            if rotor.name == name:
                rotor.position = Letter(position)

    def set_rotor_ringstellung(self, name: str, ringstellung: str):
        for rotor in self.rotors:
            if rotor.name == name:
                rotor.ringstellung = Letter(ringstellung)

    def __str__(self) -> str:
        reversed_rotor = list(reversed(self.rotors))
        return (f"Enigma {self.name}\n"
                f"Rotors: {'-'.join([r.name for r in reversed_rotor])}\n"
                f"Ringstellung: {'-'.join(str(r.ringstellung) for r in reversed_rotor)}\n"
                f"Positions: {'-'.join(str(r.position) for r in reversed_rotor)}\n"
                f"Plugboard: {str(self.plugboard)}")


In [ ]:
def test_hello_world():
    tests = [
        {"entry": "hello world", "result": "fsqsj fusta"},
        {"entry": "xgytk npnkq ssnxw kyf", "result": "alanm athis ontur ing"},
    ]

    for test in tests:
        p = Plugboard("qd fe rw jn il ps cm ax kg yu")

        M3 = Machine(
            "M3",
            [Rotors.M3.ETW, Rotors.M3.I, Rotors.M3.II, Rotors.M3.III, Rotors.M3.UKWC],
            p,
        )


        M3.set_rotor_position("I", "A")
        M3.set_rotor_position("II", "B")
        M3.set_rotor_position("III", "C")

        M3.set_rotor_ringstellung("I", "D")
        M3.set_rotor_ringstellung("II", "E")
        M3.set_rotor_ringstellung("III", "F")

        ciphertext = M3.encode(test["entry"])

        assert ciphertext == test["result"]

        M3.reset()

        M3.set_rotor_position("I", "A")
        M3.set_rotor_position("II", "B")
        M3.set_rotor_position("III", "C")

        M3.set_rotor_ringstellung("I", "D")
        M3.set_rotor_ringstellung("II", "E")
        M3.set_rotor_ringstellung("III", "F")

        plaintext = M3.encode(test["result"])

        assert plaintext == test["entry"]


def test_decode_M4_U534():
    # decode this : https://www.cryptomuseum.com/crypto/enigma/msg/p1030681.htm

    plugboard = Plugboard("AE BF CM DQ HU JN LX PR SZ VW")

    M4 = Machine(
        "M4",
        [
            Rotors.M4.ETW,
            Rotors.M4.VIII,
            Rotors.M4.VI,
            Rotors.M4.V,
            Rotors.M4.Beta,
            Rotors.M4.UKWC,
        ],
        plugboard,
    )

    M4.set_rotor_position("Beta", "C")
    M4.set_rotor_position("V", "D")
    M4.set_rotor_position("VI", "S")
    M4.set_rotor_position("VIII", "Z")

    M4.set_rotor_ringstellung("Beta", "E")
    M4.set_rotor_ringstellung("V", "P")
    M4.set_rotor_ringstellung("VI", "E")
    M4.set_rotor_ringstellung("VIII", "L")

    print(M4)

    ciphertext = """LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT
   GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN
   VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH
   VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO
   TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH
   DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML
   RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW"""

    plaintext = M4.encode(ciphertext, split=0, debug=False)

    assert (
        plaintext
        == """krkrallexxfolgendesistsofortbekanntzugebenxxichhabefolgelnbebefehlerhaltenxxjansterledesbisherigxnreichsmarschallsjgoeringjsetztderfuehrersieyhvrrgrzssadmiralyalsseinennachfolgereinxschriftlschevollmachtunterwegsxabsofortsollensiesaemtlichemassnahmenverfuegenydiesichausdergegenwaertigenlageergebenxgezxreichsleiteikktulpekkjbormannjxxobxdxmmmdurnhfkstxkomxadmxuuubooiexkp"""
    )


def test_double_stepping():
    plugboard = Plugboard("")

    M3 = Machine(
        "M3",
        [Rotors.M3.ETW, Rotors.M3.I, Rotors.M3.II, Rotors.M3.III, Rotors.M3.UKWC],
        plugboard,
    )

    M3.set_rotor_position("I", "Q")
    M3.set_rotor_position("II", "D")
    M3.set_rotor_position("III", "A")

    M3.encode("aaa")  # three rotation

    rotors = M3.get_position()

    assert (
        rotors[0].letter == "b" and rotors[1].letter == "f" and rotors[2].letter == "t"
    )

# http://wiki.franklinheath.co.uk/index.php/Enigma/Sample_Messages
def test_decode_M3_manual():
    M3 = Machine(
        "M3",
        [
            Rotors.M3.ETW,
            Rotors.M3.III,
            Rotors.M3.I,
            Rotors.M3.II,
            Rotor("UKWA", "EJMZALYXVBWFCRQUONTSPIKHGD", "", reflector=True),
        ],
        Plugboard("AM FI NV PS TU WZ"),
    )

    M3.set_rotor_position("II", "A")
    M3.set_rotor_position("I", "B")
    M3.set_rotor_position("III", "L")

    M3.set_rotor_ringstellung("II", string.ascii_lowercase[24 - 1])
    M3.set_rotor_ringstellung("I", string.ascii_lowercase[13 - 1])
    M3.set_rotor_ringstellung("III", string.ascii_lowercase[22 - 1])

    plaintext = M3.encode(
        "GCDSE AHUGW TQGRK VLFGX UCALX VYMIG MMNMF DXTGN VHVRM MEVOU YFZSL RHDRR XFJWC FHUHM UNZEF RDISI KBGPM YVXUZ",
        split=0,
    )

    assert (
        plaintext
        == "feindliqeinfanteriekolonnebeobaqtetxanfangsuedausgangbaerwaldexendedreikmostwaertsneustadt"
    )


def test_decode_enigma_K():
    K = Machine(
        "K",
        [
            Rotor("ETWK", "QWERTZUIOASDFGHJKPYXCVBNML", "", rotate=False),
            Rotor("KII", "CJGDPSHKTURAWZXFMYNQOBVLIE", "N"),
            Rotor("KI", "LPGSZMHAEOQKVXRFYBUTNICJDW", "Y"),
            Rotor("KII", "CJGDPSHKTURAWZXFMYNQOBVLIE", "N"),
            Rotor("UKWK", "IMETCGFRAYSQBZXWLHKDVUPOJN", "", reflector=True),
        ],
        Plugboard(""),
    )

    # 26 17 16 13
    K.set_rotor_ringstellung("KII", string.ascii_lowercase[13 - 1])
    K.set_rotor_ringstellung("KI", string.ascii_lowercase[16 - 1])
    K.set_rotor_ringstellung("KIII", string.ascii_lowercase[17 - 1])
    K.set_rotor_ringstellung("UKWK", string.ascii_lowercase[26 - 1])

    plaintext = K.encode("QSZVI DVMPN EXACM RWWXU IYOTY NGVVX DZ---", split=0)

    assert(
        plaintext == ""
    )

test_hello_world()
test_decode_M4_U534()
test_double_stepping()
test_decode_M3_manual()
# test_decode_enigma_K()

print("All tests passed")

## Enigma Code Book

Code books were used by the Germans to list all the settings needed to set up the Enigma machines before starting to encrypt or decrypt messages. The Germans used to change the Enigma settings very regularly (e.g. once a day) so that if the Allies managed to break their code (find out the Enigma settings) they would only be able to use them for that day and would have to find the new settings every day. Code books were highly confidential documents as if a codebook was captured or reconstructed, messages could easily be decrypted.

An Enigma code book would have one page per month. The page would include all the settings for each day of the month with the first day of the month at the bottom of the page so that once used, a setting could be torn off the page.

The settings would indicate which rotors to use and in which order to connect them. Initially the Enigma machine came with a box of five rotors to choose from. On an Enigma M3, three out of the five rotors were connected. The M4 Enigma used four rotors chosen from a box of up to eight rotors.

The settings would also include the wheel settings (how to connect the rotors) and their initial position. Finally the settings would indicate which letters to connect by plugging cables on the plugboard.

**Enigma M3 Code Book (UKW-B Reflector) April 1940**

<p align="left">
<img src="../images/mission-x-challenge/intro-M3-codebook.jpeg" alt="Enigma M3 Code Book (UKW-B Reflector) April 1940" width="480"/>  
</p>

In [ ]:
M3 = Machine(
    "M3",
    [
        Rotors.M3.ETW,
        Rotors.M3.V,
        Rotors.M3.III,
        Rotors.M3.II,
        Rotors.M3.UKWB,
    ],
    plugboard = Plugboard("AO HI MU SN VX ZQ"),
)

def init_Enigma(M):
    M.reset()
    M.set_rotor_position("V", "F")
    M.set_rotor_position("III", "D")
    M.set_rotor_position("II", "V")

    M.set_rotor_ringstellung("V", "A")
    M.set_rotor_ringstellung("III", "K")
    M.set_rotor_ringstellung("II", "K")  
    print(M)
    return M

print('-'*79)
M3 = init_Enigma(M3)
plaintext = """HELLO WORLD"""
plaintext = plaintext.upper().replace(" ", "")
ciphertext = M3.encode(plaintext, split=0, debug=True)

print(f"{plaintext.upper()  =}")
print(f"{ciphertext.upper() =}")


In [ ]:
print('-'*79)
M3 = init_Enigma(M3)
message = M3.encode(ciphertext, split=0, debug=True)
print('-'*79)
print(f"{ciphertext.upper() =}")
print(f"{message.upper()    =}")


#  German Translation Book

Code breakers working at Bletchley Park had to decrypt German communications. Some understanding of the German language was needed to be able to understand the decrypted messages. Here are a few words or expressions that were often used in German communications.
 
<p align="left">
<img src="../images/mission-x-challenge/intro-German-translation-book.png" alt="German Translation Book" width="480"/>  
</p>

```text
Kriegsmarine: German Navy
U-Boot: U-Boat
Keine besonderen Ereignisse: Nothing to report
Warten auf Anweisungen: Waiting for instructions
Numbers from 1 to 10: Eins, Zwei, Drei, Vier, Fünf, Sechs, Sieben, Acht, Neun, Zehn.
Days of the week (Monday to Sunday): Montag - Dienstag - Mittwoch - Donnerstag - Freitag - Samstag - Sonntag

Wir sind: We are / Wir werden: We will / Wir haben: We have
Angreifen: To Attack
Ziel: Target / Ziel zerstört: Target Destroyed
Hafen: Port
Nächsten: Next / Von: Of
Wettervorhersage: Weather forecast

```


# Mission X

Dear code breaker,

I am contacting you from Bletchley Park as we are intercepting an increased volume of encrypted radio signals. We are working day and night to break the Enigma codes that the German Navy are updating every day. The German U-Boats are operating in the Atlantic Ocean and are causing major casualties by sinking not only military ships from the Royal Navy but also merchant ships bringing supplies from the United States and Canada to the United Kingdom.

Luckily, we have gained access to a code book which, we believe gives us all the settings for the Enigma M3 machine, for each day of this month.

Could you help us decode the following secret messages by applying the required settings to the Enigma M3 machine. I am not sure about your understanding of the German language so I have enclosed a German translation book including key German expressions.

We believe that the Germans are preparing a major attack on the south coast of England. We hope that the messages will help us find more information on the exact location and day of the attack.

Let us know if you can find out more useful information from these messages.

Yours sincerely,

Your friend, Alan T.

<p align="left">
<img src="../images/mission-x-challenge/intro-mission-X-mail.jpeg" alt="Alan T.'s Email" width="480"/>  
</p>

## Cipher Text

- Message #1 - Received on 7 April 1940: SYTUN TJBFC BRSSY XNPKQ QZMDW
- Message #2 - Received on 11 April 1940:SDPNV OLDCQ HIBMK QTONC VNZXU SKSPZ IW
- Message #3 - Received on 12 April 1940:OJSBI BUPKA ECMEE ZH
- Message #4 - Received on 12 April 1940:REVNU XWYCV HZFSH NFMSP


<p align="left">
<img src="../images/mission-x-challenge/intro-message.jpeg" alt="Messages Ciphertext" width="480"/>  
</p>

In [ ]:
import pandas as pd

df_Mission_X = pd.DataFrame(
    {
    'Date': ['7 April 1940', '11 April 1940', '12 April 1940', '12 April 1940'],
    'Ciphertext': ['SYTUN TJBFC BRSSY XNPKQ QZMDW', 'SDPNV OLDCQ HIBMK QTONC VNZXU SKSPZ IW', 'OJSBI BUPKA ECMEE ZH', 'REVNU XWYCV HZFSH NFMSP']
    }
)
df_Mission_X['Date'] = pd.to_datetime(df_Mission_X['Date'])
# display(df_Mission_X)

df_Enigma_M3_Code_Book = pd.read_csv('../documents/Enigma_M3_Code_Book.csv', skiprows=1)
df_Enigma_M3_Code_Book['Date'] = df_Enigma_M3_Code_Book['Date'].apply(lambda x: f"1940-04-{x:02d}")
df_Enigma_M3_Code_Book['Date'] = pd.to_datetime(df_Enigma_M3_Code_Book['Date'])
# display(df_Enigma_M3_Code_Book)

df_Mission_X_Enigma_M3_Code_Book = df_Mission_X.merge(df_Enigma_M3_Code_Book, on='Date', how='left')
display(df_Mission_X_Enigma_M3_Code_Book)

In [ ]:
def Enigma_M3_Assemble(
        Rotors_selection,
        Ring_settings,
        Plugboard_settings,
        Initial_rotor_positions,
) -> Machine:
    
    Rotor_order = Rotors_selection.split(' ')
    Rotor_order.reverse() # That is how they are assembled, from right to left 
    Rotor_list = [Rotors.M3.ETW]
    for rotor in Rotor_order:
        Rotor_list.append(getattr(Rotors.M3, rotor))
    Rotor_list.append(Rotors.M3.UKWB)
    M = Machine(
        "M",
        Rotor_list,
        plugboard = Plugboard(str(Plugboard_settings)),
    )

    M.reset() 
    Rotor_order.reverse() # remember to reverse it back to mapping the ring and initial positions.
    for rotor_id, ring_set in zip(Rotor_order, Ring_settings):
        M.set_rotor_ringstellung(rotor_id, ring_set)
    for rotor_id, init_pos in zip(Rotor_order, Initial_rotor_positions):
        M.set_rotor_position(rotor_id, init_pos)

    return M
 
def test_Enigma_M3_Assemble(Rotors_selection = 'III I II',
            Ring_settings='CKU',
            Plugboard_settings='CK IZ QT NP JY GW',
            Initial_rotor_positions='VQN',
            debug=False): 
    # 'III I II', 'CKU', 'CK IZ QT NP JY GW', 'VQN'
    print('-'*79)
    M3_A = Enigma_M3_Assemble(Rotors_selection,
            Ring_settings,
            Plugboard_settings,
            Initial_rotor_positions)

    plaintext = """ZIELH AFENV ONDOV ER""" # 1940-04-12: Target Port of DOVER
    plaintext = plaintext.upper()
    ciphertext = M3_A.encode(plaintext, split=5, debug=debug).upper()
    assert ciphertext == "OJSBI BUPKA ECMEE ZH", f"Error: {ciphertext} != OJSBI BUPKA ECMEE ZH"

    print(f"{plaintext.upper()  =}")
    print(f"{ciphertext.upper() =}")
    
    M3_B = Enigma_M3_Assemble(Rotors_selection,
            Ring_settings,
            Plugboard_settings,
            Initial_rotor_positions)
    message = M3_B.encode(ciphertext, split=5, debug=debug).upper()
    print('-'*79)
    print(f"{ciphertext.upper() =}")
    print(f"{message.upper()    =}")
    assert message == plaintext, f"Error: {message} != {plaintext}"

# 1940-04-12: 
# plaintext: ZIELH AFENV ONDOV ER
# ciphertext: OJSBI BUPKA ECMEE ZH
# M3('III I II', 'CKU', 'CK IZ QT NP JY GW', 'VQN')
test_Enigma_M3_Assemble(debug=False) 

In [ ]:
display(df_Mission_X_Enigma_M3_Code_Book)

In [ ]:
# loop over the df_Mission_X_Enigma_M3_Code_Book by row 
Messages = []
for index, row in df_Mission_X_Enigma_M3_Code_Book.iterrows():
    Rotors_selection = row['Rotors']
    Ring_settings = row['Ring_settings']
    Plugboard_settings = row['Plugboard_settings']
    Initial_rotor_positions = row['Initial_rotor_positions']

    M3 = Enigma_M3_Assemble(Rotors_selection,
            Ring_settings,
            Plugboard_settings,
            Initial_rotor_positions)
    
    ciphertext = row['Ciphertext']
    message = M3.encode(ciphertext, split=5, debug=False).upper()
    Messages.append(message)

# save results to df_Mission_X_Message
df_Mission_X_Message = df_Mission_X_Enigma_M3_Code_Book[['Date','Ciphertext']].copy()
df_Mission_X_Message['Message_German'] = Messages
display(df_Mission_X_Message)
df_Mission_X_Message.to_csv('df_Mission_X_Message.csv', index=False)


The results should be like follows:

| Date       | Ciphertext                             | Message                                |
|------------|----------------------------------------|----------------------------------------|
| 1940-04-07 | SYTUN TJBFC BRSSY XNPKQ QZMDW          | KEINE BESON DEREN EREIG NISSE          |
| 1940-04-11 | SDPNV OLDCQ HIBMK QTONC VNZXU SKSPZ IW | WIRWE RDENN ACHST ENMON TAGAN GREIF EN |
| 1940-04-12 | OJSBI BUPKA ECMEE ZH                   | ZIELH AFENV ONDOV ER                   |
| 1940-04-12 | REVNU XWYCV HZFSH NFMSP                | WARTE NAUFA NWEIS UNGEN                |

By manually comparing the Message with the German Translation Book.
We will have results like  

| Date       | Ciphertext                             | Message_German                       | Translation_English           |
|------------|----------------------------------------|--------------------------------------|-------------------------------|
| 1940-04-07 | SYTUN TJBFC BRSSY XNPKQ QZMDW          | Keine besonderen ereignisse          | Noting to report              |
| 1940-04-11 | SDPNV OLDCQ HIBMK QTONC VNZXU SKSPZ IW | Wir werden nachsten montag angreifen | We will next Monday to attack |
| 1940-04-12 | OJSBI BUPKA ECMEE ZH                   | Ziel hafen von DOVER                 | Target port of DOVER          |
| 1940-04-12 | REVNU XWYCV HZFSH NFMSP                | Warten auf Anweisungen               | Waiting for instructions      |

 
<p align="left">
<img src="../images/mission-x-challenge/intro-German-translation-book.png" alt="German Translation Book" width="480"/>  
</p>


Source Code Credits belongs - [Pypi: enigma-cipher-machine ](https://pypi.org/project/enigma-cipher-machine/)

I made small changes to suit the notebooks.

Reference: https://www.101computing.net/enigma/ 


<p align="left">
<img src="../images/turing-welchman-bombe-simulator/header-Bletchley-park.jpeg" alt="Bletchley Park" width="1080"/>  
</p>


END